In [1]:
# Install dependencies
!pip install -U ultralytics
!pip install opencv-python-headless
!pip install albumentations
!pip install labelImg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.8 MB/s  0:00:00? eta -:--:--
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.236
    Uninstalling ultralytics-8.3.236:
      Successfully uninstalled ultralytics-8.3.236


In [3]:
import torch
from ultralytics import YOLO
import shutil
import pandas as pd


In [4]:
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

CUDA: True
GPU: NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [5]:
DATASET_PATH = "/home/muliaandiki/project/NutriPlate/preprocessing/data/Dietary App Segment.v1i.yolov8"
DATA_YAML = f"{DATASET_PATH}/data.yaml"

BASE_MODEL = "yolov8s.pt" 

In [6]:
model = YOLO(BASE_MODEL)
model.info()

YOLOv8s summary: 129 layers, 11,166,560 parameters, 0 gradients, 28.8 GFLOPs


(129, 11166560, 0, 28.816844800000002)

In [7]:
model.train(
    data=DATA_YAML,
    epochs=100,
    imgsz=640,
    batch=4,              
    optimizer="AdamW",
    patience=20,          
    device=0,
    project="runs/train",
    name="dietary_yolov8s",
    exist_ok=True,
    workers=4,
    verbose=True
)

Ultralytics 8.3.240 🚀 Python-3.12.3 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 5804MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/muliaandiki/project/NutriPlate/preprocessing/data/Dietary App Segment.v1i.yolov8/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=dietary_yolov8s, nbs=64, nms=False, opset=None

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ca1d4150ef0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04104

In [8]:
best_model = YOLO("runs/train/dietary_yolov8s/weights/best.pt")

metrics = best_model.val(
    data=DATA_YAML,
    split="val",
    imgsz=640,
    device=0
)

print(metrics)

Ultralytics 8.3.240 🚀 Python-3.12.3 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 5804MiB)
Model summary (fused): 72 layers, 11,135,646 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2303.3±1312.7 MB/s, size: 58.2 KB)
val: Scanning /home/muliaandiki/project/NutriPlate/preprocessing/data/Dietary App Segment.v1i.yolov8/valid/labels.cache... 678 images, 1 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 678/678 1.4Mit/s 0.0s0s
val: /home/muliaandiki/project/NutriPlate/preprocessing/data/Dietary App Segment.v1i.yolov8/valid/images/Image_43_jpg.rf.04a45ce90074bb4924a043bb0c08f5db.jpg: 1 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 43/43 4.7it/s 9.1s0.2s
                   all        678       1709      0.736      0.712      0.755       0.64
            ayam bakar         13         25      0.757        0.6      0.758      0.591
           ayam goreng  

In [9]:

df = pd.DataFrame({
    "Class": metrics.names.values(),
    "Precision": metrics.box.p,
    "Recall": metrics.box.r,
    "mAP50": metrics.box.map50,
    "mAP50-95": metrics.box.map
})

df.sort_values("mAP50", ascending=False)


,Class,Precision,Recall,mAP50,mAP50-95
0,ayam bakar,0.756581,0.600000,0.755034,0.640403
1,ayam goreng,0.587664,0.375122,0.755034,0.640403
2,bakso,0.764685,0.828729,0.755034,0.640403
3,bakwan,0.644654,0.426471,0.755034,0.640403
4,bihun,0.863717,0.905661,0.755034,0.640403
5,capcay,0.916875,0.888889,0.755034,0.640403
6,gado-gado,0.677429,0.820513,0.755034,0.640403
7,ikan goreng,0.615179,0.711111,0.755034,0.640403
8,kerupuk,0.821420,0.651314,0.755034,0.640403
9,martabak telur,0.812041,0.769231,0.755034,0.640403


In [10]:
TEST_IMAGES = f"{DATASET_PATH}/test/images"

results = best_model.predict(
    source=TEST_IMAGES,
    imgsz=640,
    conf=0.25,
    iou=0.5,
    save=True,
    device=0
)



image 1/400 /home/muliaandiki/project/NutriPlate/preprocessing/data/Dietary App Segment.v1i.yolov8/test/images/02-resep_tumis_kangkung_-bumbu_dan_cara_membuat_atau_cara_memasak_jpg.rf.7f273a73b570d6619340918daa9f9d40.jpg: 640x640 1 tumis kangkung, 12.5ms
image 2/400 /home/muliaandiki/project/NutriPlate/preprocessing/data/Dietary App Segment.v1i.yolov8/test/images/1-resepmasakterbaru_jpg.rf.272cd0552262bb9bd0e0dff6d6551d1c.jpg: 640x640 1 gado-gado, 11.3ms
image 3/400 /home/muliaandiki/project/NutriPlate/preprocessing/data/Dietary App Segment.v1i.yolov8/test/images/1000_F_401432670_M35EYFpGInyJNF2WFg0mc1lniH3RU1Mh_jpg.rf.df870029bf7fe2d103035091c0b6cbaa.jpg: 640x640 6 kerupuks, 11.9ms
image 4/400 /home/muliaandiki/project/NutriPlate/preprocessing/data/Dietary App Segment.v1i.yolov8/test/images/10_jpg.rf.60c5d389c54d33f8b77e4131af4d4707.jpg: 640x640 1 martabak telur, 12.2ms
image 5/400 /home/muliaandiki/project/NutriPlate/preprocessing/data/Dietary App Segment.v1i.yolov8/test/images/1113

In [11]:
best_model.val(
    data=DATA_YAML,
    plots=True


SyntaxError: incomplete input (3271625816.py, line 3)

In [ ]:
shutil.copy(
    "runs/train/dietary_yolov8s/weights/best.pt",
    "dietary_food_detector.pt"
)

In [ ]:
best_model.export(format="torchscript")
best_model.export(format="onnx")